# CSE 482 - Project

In [1]:
import pandas as pd
from sodapy import Socrata
import matplotlib.pyplot as plt
%matplotlib inline

In [9]:
myAppToken = 'rEPOXGSegb5AUSutACzeimRkD'
client = Socrata('data.cityofchicago.org', myAppToken)

results = client.get('ijzp-q8t2', limit=100000)
df = pd.DataFrame.from_records(results)

In [18]:
data = df.drop(columns=[':@computed_region_awaf_s7ux', ':@computed_region_6mkv_f3dw', ':@computed_region_vrxf_vc4k', ':@computed_region_bdys_3d7i', 
                      ':@computed_region_43wa_7qmu', ':@computed_region_rpca_8um6', ':@computed_region_d9mm_jgwp', ':@computed_region_d3ds_rm58',
                      'id', 'case_number', 'block', 'iucr', 'domestic', 'beat', 'ward', 'community_area', 'fbi_code', 'x_coordinate', 'y_coordinate',
                       'updated_on', 'latitude', 'longitude', 'location'])
data.head()

,date,primary_type,description,location_description,arrest,district,year
0,2020-04-16T23:55:00.000,ASSAULT,AGGRAVATED - HANDGUN,RESIDENCE,False,007,2020
1,2020-04-16T23:55:00.000,BATTERY,AGGRAVATED - KNIFE / CUTTING INSTRUMENT,APARTMENT,False,003,2020
2,2020-04-16T23:55:00.000,BATTERY,SIMPLE,PARKING LOT / GARAGE (NON RESIDENTIAL),False,009,2020
3,2020-04-16T23:50:00.000,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,017,2020
4,2020-04-16T23:50:00.000,BATTERY,DOMESTIC BATTERY SIMPLE,APARTMENT,False,002,2020
